In [75]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import os
from timeit import default_timer as dtimer

In [76]:
# Load the data for each datafile into a dataframe
df = pd.DataFrame()
for filename in os.listdir("data"):
    if "data_" in filename:
        t1 = dtimer()
        temp_df = pd.read_csv(os.path.join("data",filename))
        df = pd.concat([df, temp_df], axis=1)
        t2 = dtimer()
        print("Added {}, time={}s".format(filename, round(t2-t1, 2)))

print("Complete.")
df.head()
        
        
# mdf = pd.DataFrame()
# idf = pd.read_csv("data/data_1.csv")

Added data_1.csv, time=5.89s
Added data_2.csv, time=2.85s
Added data_2000-2002_modules.csv, time=3.76s
Added data_2004_modules.csv, time=4.1s
Added data_2006-2010_modules.csv, time=6.77s
Added data_2012-2014_modules.csv, time=3.32s
Added data_3.csv, time=9.69s
Added data_4.csv, time=9.95s
Added data_80s-90s_issp_modules.csv, time=10.21s
Added data_80s-90s_modules.csv, time=11.08s
Complete.


,CASEID,YEAR,ID,AGE,SEX,RACE,RACECEN1,RACECEN2,RACECEN3,HISPANIC,...,WORKUNDC,OBRESPCT,ECONPAST,PASTUP,PASTDOWN,ECONFUTR,FUTRUP,FUTRDOWN,RDISCAFF,RIMMDISC
0,1972 1,1972,1,23,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1972 2,1972,2,70,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1972 3,1972,3,48,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1972 4,1972,4,27,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1972 5,1972,5,61,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
def compile_variable_dict(ddi_filepath):
    root = ET.parse(ddi_filepath)
    data = root.find("dataDscr")
    
    var_dict = {}
    for var in data:
        # Get the ID Code of the variable
        id_code = var.get("ID")
        # Get the Full Name of the variable
        full_title = var.find("./labl").text.strip()
        # Get the Question Text (if applicable)
        qstn = var.find("qstn")
        if qstn != None:
            question_text = qstn.find("qstnLit").text.strip()
        else:
            question_text = None
        # Get the categories
        category_dict = {}
        category_list = var.findall("catgry")
        for category in category_list:
            value = float(category.find("catValu").text.strip())
            label = category.find("*[@level='category']").text.strip()
            missing = category.get("missing")
            category_dict[value] = label
            # category_dict[value] = {"label":label, "missing":missing}
            
        var_dict[id_code] = {
            "title": full_title,
            "question_text": question_text,
            "categories": category_dict
        }
    
    return var_dict

In [84]:
import json
var_dict = {}
for filename in os.listdir("data"):
    if "ddi_" in filename:
        root = ET.parse(os.path.join("data", filename))
        data = root.find("dataDscr")
        for var in data:
            var_code = var.get("ID")
            var_dict[var_code] = filename

json.dump(var_dict, open("data/_var_lookup.json", "w"), indent=4)

In [82]:
for filename in os.listdir("data"):
    if "ddi_" in filename:
        t1 = dtimer()
        filepath = os.path.join("data", filename)
        var_dict = compile_variable_dict(filepath)
#         print(var_dict)
        for col in df.columns:
            if col in var_dict:
                try:
                    df[col] = df[col].astype(float).replace(to_replace=var_dict[col]['categories'], inplace=True)
                except Exception as e:
                    print(e)
                    print("Variable '{}' not found in dataframe".format(var))
        t2 = dtimer()
        print("Translated {}, time={}s".format(filename, round(t2-t1, 2)))
print("Complete.")
df.head()

could not convert string to float: '1972 1  '
Variable 'HELPBLK' not found in dataframe


KeyboardInterrupt: 

In [79]:
df.head()

,CASEID,YEAR,ID,AGE,SEX,RACE,RACECEN1,RACECEN2,RACECEN3,HISPANIC,...,WORKUNDC,OBRESPCT,ECONPAST,PASTUP,PASTDOWN,ECONFUTR,FUTRUP,FUTRDOWN,RDISCAFF,RIMMDISC
0,1972 1,1972,1,23,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1972 2,1972,2,70,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1972 3,1972,3,48,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1972 4,1972,4,27,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1972 5,1972,5,61,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
idf.dtypes[idf.dtypes==np.object]

CASEID    object
dtype: object

In [30]:
df = pd.read_csv("data/data_1.csv")

In [31]:
df.head()

,CASEID,YEAR,ID,AGE,SEX,RACE,RACECEN1,RACECEN2,RACECEN3,HISPANIC,...,LIFE,HELPFUL,FAIR,TRUST,TRUSTY,SATCITY,SATHOBBY,SATFAM,SATFRND,SATHEALT
0,1972 1,1972,1,23,2,1,0,0,0,0,...,0,2,2,3,0,0,0,0,0,0
1,1972 2,1972,2,70,1,1,0,0,0,0,...,0,1,2,1,0,0,0,0,0,0
2,1972 3,1972,3,48,2,1,0,0,0,0,...,0,2,1,2,0,0,0,0,0,0
3,1972 4,1972,4,27,2,1,0,0,0,0,...,0,2,2,2,0,0,0,0,0,0
4,1972 5,1972,5,61,2,1,0,0,0,0,...,0,2,2,2,0,0,0,0,0,0


In [79]:
root = ET.parse('data/sub-ddi.txt').getroot()

data = root.find("dataDscr")

var_dict = {}
for var in data:
    # Get the ID Code of the column
    id_code = var.get("ID")
    # Get the Full Name of the column
    full_title = var.find("./labl").text.strip()
    # Get the Question Text (if applicable)
    qstn = var.find("qstn")
    if qstn != None:
        qlit = qstn.find("qstnLit").text
        question_text = qlit.strip()#.replace("\n", "")
    else:
        question_text = None
    
    # Get the categories
    category_dict = {}
    category_list = var.findall("catgry")
    for category in category_list:
        value = float(category.find("catValu").text.strip())
        label = category.find("*[@level='category']").text.strip()
        missing = category.get("missing")
        category_dict[value] = label
#         category_dict[value] = {"label":label, "missing":missing}
    
    # Compile into dictionary
    var_dict[id_code] = {
        "title": full_title,
        "question_text": question_text,
        "categories": category_dict
    }

In [34]:
def compile_variable_dict(ddi_filepath):
    root = ET.parse(ddi_filepath)
    data = root.find("dataDscr")
    
    var_dict = {}
    for var in data:
        # Get the ID Code of the variable
        id_code = var.get("ID")
        # Get the Full Name of the variable
        full_title = var.find("./labl").text.strip()
        # Get the Question Text (if applicable)
        qstn = var.find("qstn")
        if qstn != None:
            question_text = qstn.find("qstnLit").text.strip()
        else:
            question_text = None
        # Get the categories
        category_dict = {}
        category_list = var.findall("catgry")
        for category in category_list:
            value = float(category.find("catValu").text.strip())
            label = category.find("*[@level='category']").text.strip()
            missing = category.get("missing")
            category_dict[value] = label
            # category_dict[value] = {"label":label, "missing":missing}
            
        var_dict[id_code] = {
            "title": full_title,
            "question_text": question_text,
            "categories": category_dict
        }
    
    return var_dict

var_dict = compile_variable_dict('data/ddi_1.xml')

In [35]:
value_df = df.copy()

for col in value_df.columns:
    try:
        value_df[col].replace(to_replace=var_dict[col]['categories'], inplace=True)
    except AttributeError as e:
        print(e)
        print(var)

In [36]:
value_df.head()

,CASEID,YEAR,ID,AGE,SEX,RACE,RACECEN1,RACECEN2,RACECEN3,HISPANIC,...,LIFE,HELPFUL,FAIR,TRUST,TRUSTY,SATCITY,SATHOBBY,SATFAM,SATFRND,SATHEALT
0,1972 1,1972,1,23,FEMALE,WHITE,IAP,IAP,IAP,IAP,...,IAP,LOOKOUT FOR SELF,FAIR,DEPENDS,IAP,IAP,IAP,IAP,IAP,IAP
1,1972 2,1972,2,70,MALE,WHITE,IAP,IAP,IAP,IAP,...,IAP,HELPFUL,FAIR,CAN TRUST,IAP,IAP,IAP,IAP,IAP,IAP
2,1972 3,1972,3,48,FEMALE,WHITE,IAP,IAP,IAP,IAP,...,IAP,LOOKOUT FOR SELF,TAKE ADVANTAGE,Can't be too careful,IAP,IAP,IAP,IAP,IAP,IAP
3,1972 4,1972,4,27,FEMALE,WHITE,IAP,IAP,IAP,IAP,...,IAP,LOOKOUT FOR SELF,FAIR,Can't be too careful,IAP,IAP,IAP,IAP,IAP,IAP
4,1972 5,1972,5,61,FEMALE,WHITE,IAP,IAP,IAP,IAP,...,IAP,LOOKOUT FOR SELF,FAIR,Can't be too careful,IAP,IAP,IAP,IAP,IAP,IAP


In [69]:
for var in var_dict:
    cat_dict = var_dict[var]['categories']
    cat_set = set()
    for cat in cat_dict:
        label = cat_dict[cat]['label']
        if label in cat_set:
            print("ERROR: VAR={}, Cat={}, Label={}".format(var, cat, label))
        else:
            cat_set.add(label)

ERROR: VAR=HHTYPE, Cat=26, Label=3ADLTS,0MAR,REL,0KIDS
ERROR: VAR=HHTYPE, Cat=46, Label=3ADLTS,2MAR,REL,NTREL,0KIDS
ERROR: VAR=HHTYPE, Cat=52, Label=3ADLTS,3MAR,SP,REL,0KIDS
ERROR: VAR=HHTYPE, Cat=126, Label=3ADLTS,0MAR,REL,1+KIDS
ERROR: VAR=HHTYPE, Cat=146, Label=3ADLTS,2MAR,REL,NTREL,1+KIDS
ERROR: VAR=HHTYPE, Cat=152, Label=3ADLTS,3MAR,SP,REL,1+KIDS
ERROR: VAR=PAIND16, Cat=999, Label=NA
ERROR: VAR=OTHER, Cat=165, Label=Church of the Living God
ERROR: VAR=OTHER, Cat=209, Label=Church of the Living God
ERROR: VAR=OTH16, Cat=165, Label=Church of the Living God
ERROR: VAR=OTH16, Cat=209, Label=Church of the Living God
ERROR: VAR=SPOTHER, Cat=165, Label=Church of the Living God
ERROR: VAR=SPOTHER, Cat=209, Label=Church of the Living God
ERROR: VAR=SPOTH16, Cat=165, Label=Church of the Living God


In [74]:
df[['AGE','JOBYEARS']].dtypes

AGE           int64
JOBYEARS    float64
dtype: object